In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np
import time

In [ ]:
data_file = open("dataset.txt", "r")
lines = data_file.readlines()

def func_replacer(line):
    line = line
    if "sin" in line or "cos" in line or "tan" in line:
        line = line.replace("sin", "!")
        line = line.replace("cos", "@")
        line = line.replace("tan", "#")
    return line

letter_to_idx = {}
idx_to_letter = {}
cnt = 0
for i in range(10):
    letter_to_idx[str(i)] = len(letter_to_idx)
for line in lines:
    line = func_replacer(line)
    for letter in line: 
        if letter.isalpha(): letter = "$";
        if letter not in letter_to_idx and letter != "=": letter_to_idx[letter] = len(letter_to_idx);

processed_lines = []
max_digit = 0
max_num = 0
for line in lines:
    line = func_replacer(line)
    line = line.replace("\n", "")
    left = line.split("=")[0]
    right = line.split("=")[1]
    processed_line_0 = []
    processed_line_1 = []
    for i, letter in enumerate(left):
        processed_letter = None
        if letter.isnumeric():
            j = i+1; digit = 0
            while j < len(left):
                if left[j].isnumeric(): 
                    digit += 1
                    if digit > max_digit: max_digit = digit
                else: break
                j += 1
            processed_letter = str(int(letter)*(10**digit))
            processed_line_0.append( processed_letter )
        else: 
            processed_letter = letter
            processed_line_0.append( processed_letter )
        if processed_letter not in letter_to_idx and letter != "=": letter_to_idx[processed_letter] = len(letter_to_idx);

    for i, letter in enumerate(right):
        processed_letter = None
        if letter.isnumeric():
            j = i+1; digit = 0
            while j < len(right):
                if right[j].isnumeric(): 
                    digit += 1
                    if digit > max_digit: max_digit = digit
                else: break
                j += 1
            processed_letter = str(int(letter)*(10**digit))
            processed_line_1.append( processed_letter )
        else: 
            processed_letter = letter
            processed_line_1.append( processed_letter )
        if processed_letter not in letter_to_idx and letter != "=": letter_to_idx[processed_letter] = len(letter_to_idx);

    processed_lines.append( (processed_line_0,processed_line_1))

for k,v in letter_to_idx.items(): idx_to_letter[v] = k

random.shuffle(processed_lines)
train_lines = processed_lines[: int(len(lines)*0.99) ]
test_lines = processed_lines[int(len(lines)*0.99): ]
print(len(train_lines),len(test_lines))
print(cnt)
print(letter_to_idx)
print(idx_to_letter)
print(len(lines))
idx_to_letter[52] = "START"; idx_to_letter[53] = "END"

990000 10000
0
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '(': 10, '-': 11, '*': 12, '$': 13, ')': 14, '\n': 15, '+': 16, '@': 17, '#': 18, '!': 19, 'z': 20, '10': 21, '60': 22, 's': 23, 'n': 24, 'x': 25, '20': 26, '90': 27, 'c': 28, '100': 29, 'k': 30, '200': 31, '40': 32, 'j': 33, 'h': 34, '50': 35, 'y': 36, '30': 37, '300': 38, '80': 39, 'i': 40, 'o': 41, '400': 42, 'a': 43, '700': 44, '600': 45, '70': 46, 't': 47, '900': 48, '500': 49, '800': 50, '1000': 51}
{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '(', 11: '-', 12: '*', 13: '$', 14: ')', 15: '\n', 16: '+', 17: '@', 18: '#', 19: '!', 20: 'z', 21: '10', 22: '60', 23: 's', 24: 'n', 25: 'x', 26: '20', 27: '90', 28: 'c', 29: '100', 30: 'k', 31: '200', 32: '40', 33: 'j', 34: 'h', 35: '50', 36: 'y', 37: '30', 38: '300', 39: '80', 40: 'i', 41: 'o', 42: '400', 43: 'a', 44: '700', 45: '600', 46: '70', 47: 't', 48: '900', 49: '500', 50: '800', 51: '1000'}
10000

In [ ]:
# test = tensor_preparer(['(-2*@(n)-6)*(7*@(n)+13)=-14*@(n)**2-68*@(n)-78\n'])[0][0].view(-1).cpu().detach().numpy()
# print(test)
# tokens_2_lines(test)
# print(test_lines[:1000])
print(letter_to_idx.keys())
print(len(letter_to_idx))
print(test_lines[0])

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '(', '-', '*', '$', ')', '\n', '+', '@', '#', '!', 'z', '10', '60', 's', 'n', 'x', '20', '90', 'c', '100', 'k', '200', '40', 'j', 'h', '50', 'y', '30', '300', '80', 'i', 'o', '400', 'a', '700', '600', '70', 't', '900', '500', '800', '1000'])
52
(['-', '9', '*', 'i', '*', '(', 'i', '+', '10', '3', ')'], ['-', '9', '*', 'i', '*', '*', '2', '-', '100', '10', '7', '*', 'i'])


In [ ]:

device = torch.device("cuda")
def line_2_tokens(line):
    tokens = [ letter_to_idx[letter] if not letter.isalpha() else letter_to_idx["$"] for letter in line]
    return tokens

def tokens_2_lines(line):
    tokens = [ idx_to_letter[letter] for letter in line]
    return tokens
def tensor_preparer(lines):
    tensor_lines = []
    for line in lines:
        encoder_tokens = torch.tensor( line_2_tokens( line[0] ) ).view(-1,1).to(device)
        decoder_tokens = torch.tensor( line_2_tokens( line[1] ) + [len(letter_to_idx)+1] ).view(-1,1).to(device)
        tensor_lines.append((encoder_tokens, decoder_tokens))
    return tensor_lines

In [ ]:
train_tensors = tensor_preparer(train_lines)
test_tensors = tensor_preparer(test_lines)

In [ ]:
MAX_LENGTH = 29
SOS_token = len(letter_to_idx)
EOS_token = len(letter_to_idx) + 1

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers=3)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(3, 1, self.hidden_size, device=device)

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size, num_layers=3)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(3, 1, self.hidden_size, device=device)

In [ ]:
hidden_size = 256
vocab_size = len(letter_to_idx)
encoder = EncoderRNN(vocab_size, hidden_size).to(device)
attn_decoder = AttnDecoderRNN(hidden_size, vocab_size + 2, dropout_p=0.1).to(device)

In [ ]:
learning_rate = 0.01
encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(attn_decoder.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

In [ ]:
def evaluate(encoder, decoder, max_length=MAX_LENGTH):
    total_instances = instance_correct = token_correct = total_token = 0;
    with torch.no_grad():
        for i, pair in enumerate(test_tensors):
            encoder_hidden = encoder.initHidden()
            input_tensor = pair[0]
            target_tensor = pair[1]
            input_length = input_tensor.size(0)
            target_length = target_tensor.size(0)
            encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
            loss = 0

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(
                    input_tensor[ei], encoder_hidden)
                encoder_outputs[ei] = encoder_output[0, 0]

            decoder_input = torch.tensor([[SOS_token]], device=device)

            decoder_hidden = encoder_hidden

            loss = 0

            preds = []

            for di in range(target_length):
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
                pred = np.argmax(decoder_output.cpu().detach().numpy(), axis=-1)[0]
                preds.append(pred)
                loss += criterion(decoder_output, target_tensor[di])
                decoder_input = target_tensor[di]
                # if decoder_input.item() == EOS_token:
                #     break
            # print(tokens_2_lines(preds))
            # print(tokens_2_lines(target_tensor.view(-1).cpu().detach().numpy()))
            # print(" ")
            sentence_correct = True
            for p, t in zip(preds, target_tensor.view(-1).cpu().detach().numpy()):
                if p == t: token_correct += 1
                else: sentence_correct = False
                total_token += 1
            if sentence_correct: instance_correct += 1
            total_instances += 1

    return total_instances, instance_correct, token_correct, total_token


def train(encoder, decoder, epoches=1):
    start = time.time()
    for i, pair in enumerate(train_tensors):

        input_tensor = pair[0]
        target_tensor = pair[1]

        encoder_hidden = encoder.initHidden()
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        input_length = input_tensor.size(0)
        target_length = target_tensor.size(0)
        encoder_outputs = torch.zeros(MAX_LENGTH, encoder.hidden_size, device=device)
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(
                input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]
        
        loss = 0
        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden

        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]

        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()
        if i % 20000 == 0: 
            end = time.time()
            print(end-start, end = " ")
            start = end
            print(i/len(train_lines), end =" ")
        if i % 60000 == 0 and i != 0:
            total_instances, instance_correct, token_correct, total_token = evaluate(encoder, attn_decoder)
            print(" ")
            print(token_correct, total_token, token_correct/total_token, instance_correct, total_instances, instance_correct/total_instances)
            print(" ")

In [ ]:
train(encoder, attn_decoder)
# evaluate(encoder, attn_decoder)

0.0854482650756836 0.0 808.4351253509521 0.020202020202020204 807.0373070240021 0.04040404040404041 793.9648885726929 0.06060606060606061  
136996 153098 0.89482553658441 2794 10000 0.2794
 
936.5924754142761 0.08080808080808081 790.5000705718994 0.10101010101010101 798.5208611488342 0.12121212121212122  
142831 153098 0.9329383793387243 4030 10000 0.403
 
949.6776471138 0.1414141414141414 823.286376953125 0.16161616161616163 827.2908527851105 0.18181818181818182  
144454 153098 0.9435394322590759 4330 10000 0.433
 
963.3850231170654 0.20202020202020202 808.0658020973206 0.2222222222222222 

KeyboardInterrupt: ignored

In [ ]:
total_instances, instance_correct, token_correct, total_token = evaluate(encoder, attn_decoder)
print(token_correct, total_token, token_correct/total_token, instance_correct, total_instances, instance_correct/total_instances)

In [ ]:
evaluate(encoder, attn_decoder)

In [ ]:
pytorch_total_params = sum(p.numel() for p in encoder.parameters() ) #if p.requires_grad)
print(pytorch_total_params)

802816


In [ ]:
pytorch_total_params = sum(p.numel() for p in attn_decoder.parameters() ) #if p.requires_grad)
print(pytorch_total_params)

1358163
